In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [180]:
info = get_info(sfreq=100, kind="biosemi64")
fwd = create_forward_model(sampling="ico3", info=info)
leadfield = fwd["sol"]["data"]
n_chans, n_dipoles = leadfield.shape

import mne
from scipy.sparse.csgraph import laplacian

adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
laplace_operator = abs(laplacian(adjacency))
laplace_operator = laplace_operator @ laplace_operator

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.3s remaining:    3.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.3s remaining:    1.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


-- number of adjacent vertices : 1284


# Models

## The Generator

In [220]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda, LayerNormalization, Activation, Dropout, ActivityRegularization, TimeDistributed, Reshape, Permute
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
# import tensorflow_probability as tfp
from tensorflow.keras.regularizers import l1, l2, l1_l2

def weightedLoss(w):
    def loss(true, pred):
        print("DISC", true, pred)
        
        error = K.square(true - pred)
        # error = K.abs(true - pred)
        error = K.switch(K.equal(true, 0), w * error , error)
        # normalize for number of active sites
        error = error / tf.linalg.norm(true, axis=-1)
        return error
    return loss

def weightedLossGan(w):
    def loss(true, pred):
        print("GAN", true, pred)
        error = K.square(true - pred)
        # error = K.abs(true - pred)
        error = K.switch(K.equal(true, 0), w * error , error)
        # normalize for number of active sites
        error = error * tf.linalg.norm(true)
        return -error
    return loss


def custom_gan_loss(y_ones, y_both):
    y_true = y_both[0]
    y_pred = y_both[1]
    # print(y_ones, y_true, y_pred)
    
    error = -tf.keras.losses.CosineSimilarity()(y_true, y_pred)
    # error = -tf.keras.losses.mean_squared_error(y_true, y_pred)
    # blur =  tf.cast(tf.size(y_true), dtype=tf.float32) / tf.math.count_nonzero(y_true, dtype=tf.float32)
    # blur = K.mean(K.abs(y_true))
    # error = K.square(y_true -y_pred)
    # error = error / tf.linalg.norm(y_true, axis=-1)
    # error = -K.mean(error)
    # normalize for number of active sites
    # error = error / tf.linalg.norm(y_true)
    # norm = tf.linalg.norm(y_true)

    # Maximize variance of norms
    # norm_variance = 1/tf.math.reduce_std(tf.linalg.norm(y_true, axis=-1))

    return error #+ norm

def square(x):
    return K.square(x) * K.sign(x)
    
def scale_act(x):
    return tf.transpose(tf.transpose(x) / tf.math.reduce_max(K.abs(x), axis=-1))

def RelativeCosineSimilarity():
    def relative_cosine_similarity(y_true, y_pred):
        distance = tf.keras.losses.CosineSimilarity()(y_true, y_pred)
        

class CustomDropout(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training: #you can remove this line,, so that you can use dropout on inference time
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

def define_models(latent_dim, hidden_units=200, reg=1, n_dense_layers=4, activation="tanh"):
    n_chans, n_dipoles = leadfield.shape
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    laplace_operator_ = tf.cast(laplace_operator, dtype=tf.float32)
    drop_min = 0.95
    drop_max = 0.99
    # G MODEL
    # ------------------------------------------------------------------------
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    fc = Dense(latent_dim, activation=activation, name="HL_G1")(inputs)
    fc = Dropout(0.5)(fc)
    fc = Dense(latent_dim, activation=activation, name="HL_G2")(fc)
    fc = Dropout(0.5)(fc)
    # gen_out = Dense(n_dipoles, name="Output_Generator", activation=activation, activity_regularizer=l1_l2(reg))(fc)
    gen_out = Dense(n_dipoles, name="Output_Generator", activation=activation)(fc)
    # gen_out = ActivityRegularization(l2=reg)(gen_out)
    
    # gen_out = Dropout(0.97)(gen_out, training=True)
    # gen_out = Dropout(K.random_uniform((1,), drop_min, drop_max)[0])(gen_out, training=True)
    # Thresholding/ Sparsification
    # gen_out = Activation(scale_act)(gen_out)
    # gen_out = Activation("tanh")(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)==K.max(K.abs(x)), dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)>0.9, dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)>tfp.stats.percentile(K.abs(x), K.random_uniform((1,), drop_min, drop_max)), dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    
    
    # Smoothing
    gen_out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(laplace_operator_, tf.transpose(x))), output_shape=(None, n_chans))(gen_out)
    
    # Scaling
    # gen_out = LayerNormalization(center=False)(gen_out)
    gen_out = Activation(scale_act)(gen_out)
    gen_out = Activation("tanh")(gen_out)
    # Sparsify
    # gen_out = ActivityRegularization(l2=reg)(gen_out)
    g_model = tf.keras.Model(inputs=inputs, outputs=gen_out, name='Generator')
    # g_model.build(input_shape=(latent_dim))
    
    # D MODEL
    # ------------------------------------------------------------------------
    input_shape = (n_chans)
    inputs2 = tf.keras.Input(shape=input_shape, name='Input_Discriminator')
    
    fc2 = Dense(hidden_units, activation=activation, name="HL_D1")(inputs2)
    for i in range(n_dense_layers-1):
        fc2 = Dense(hidden_units, activation=activation, name=f"HL_D{i+2}")(fc2)
        
    # out = Dense(n_dipoles, activation=activation, activity_regularizer=l2(reg), name="Output_Final")(fc2)
    out_main = Dense(n_dipoles, activation=activation, name="Output_Final")(fc2)
    out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(laplace_operator_, tf.transpose(x))), output_shape=(n_dipoles))(out_main)
    out = Activation(scale_act)(out)
    out = Activation("tanh")(out)
    # out = ActivityRegularization(l2=reg)(out)
    d_model = tf.keras.Model(inputs=inputs2, outputs=out, name='Discriminator')
    # d_model.add_loss(norm_ratio_ineq(out))
    # d_model.build(input_shape=(latent_dim))
    
    # GAN MODEL
    # ------------------------------------------------------------------------
    # d_model.trainable = False
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    output_1 = g_model(inputs)
    # output_1 = Dropout(0.97)(output_1, training=True)
    # output_1 = Dropout(K.random_uniform((1,), drop_min, drop_max)[0])(output_1, training=True)
    # print("source from g: ", output_1)
    lam = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))
    eeg = lam(output_1)
    # print("eeg from source from g: ", eeg)
    eeg_normed = LayerNormalization()(eeg)
    # print("eeg_normed from source from g: ", eeg_normed)
    
    # eeg_noise = tf.keras.layers.GaussianNoise(0.2)(eeg_normed)

    output_3 = d_model(eeg_normed)
    gan_model = Model(inputs, [output_1, output_3])

    # g_model.compile(loss=custom_gan_loss, optimizer="adam")
    d_model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam", metrics=[tf.keras.losses.CosineSimilarity(),])
    # d_model.compile(loss=weightedLoss(10), optimizer="adam")
    
    # Construct your custom loss as a tensor
    # loss = tf.math.log(-tf.keras.losses.CosineSimilarity()(output_1, output_3)) + K.square(norm_ratio_ineq(output_1))/5
    loss = -tf.keras.losses.CosineSimilarity()(output_1, output_3) + l1_sparsity(output_1)*10
    
    # loss = norm_ratio_ineq(output_1)
    
    
    # loss = weightedLossGan(10)(output_1, output_3)
    
    # Add loss to model
    gan_model.add_loss(loss)

    gan_model.compile(optimizer=tf.keras.optimizers.Adam())
    # gan_model.compile(loss=custom_gan_loss, optimizer="adam")
    
    return g_model, d_model, gan_model

def norm_ratio_ineq(x):
    l1 = K.mean(K.abs(x)) 
    l2 = K.mean(K.square(x)) 
    return l1 / l2 

def l1_sparsity(x):
    return K.mean(K.abs(x)) 

def prep_data(X, y):
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)
    return X, y
    
def prep_data_sim(sim):
    X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.squeeze(np.stack([src.data for src in sim.source_data]))
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

    if len(X.shape) == 2:
        X = np.expand_dims(X, axis=-1)
        y = np.expand_dims(y, axis=-1)
    X = np.swapaxes(X, 1,2)
    y = np.swapaxes(y, 1,2)
    return X, y

def generate_samples(g_model, batch_size, latent_dim):
    x_input = np.random.randn(batch_size, latent_dim)
    sources = g_model.predict(x_input)
    return sources

n_epochs = 200
batch_size = 32
batch_number = 10
latent_dim = 16
g, d, gan = define_models(latent_dim, hidden_units=latent_dim)
g(np.random.randn(batch_size, latent_dim)).shape

TensorShape([32, 1284])

## Train

In [163]:
settings = dict(duration_of_trial=0.01, extents=(20, 30), number_of_sources=(1,15), target_snr=1e99)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=100)
X_test = np.stack([eeg.average().data[:, 0] for eeg in sim.eeg_data], axis=0)
y_test = np.stack([source.data[:, 0] for source in sim.source_data], axis=0)
X_test, y_test = prep_data(X_test, y_test)

Simulating data based on sparse patches.


100%|██████████| 100/100 [00:00<00:00, 482.50it/s]


In [221]:
from tqdm.notebook import tqdm
n_chans, n_dipoles = leadfield.shape
n_epochs = 1000
batch_size = 1024
latent_dim = 500
hidden_units = 500
reg = 0.01
gen_mod = 1
disc_mod = 1
g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg)
X_history = np.zeros((0, n_chans))
y_history = np.zeros((0, n_dipoles))
gan_losses = np.zeros(n_epochs)
d_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)
k = 20
for i in tqdm(range(n_epochs)):
    if i % disc_mod == 0:
        y = generate_samples(g_model, batch_size, latent_dim)
        X = (leadfield @ y.T).T
        X, y = prep_data(X,y)
        X_history = np.append(X_history, X, axis=0)
        y_history = np.append(y_history, y, axis=0)
        idc = np.arange(X_history.shape[0])
        np.random.shuffle(idc)
        X_history = X_history[idc]
        y_history = y_history[idc]
        X_history = X_history[:batch_size]
        y_history = y_history[:batch_size]

        # d_model.trainable = True
        # d_model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")
        d_loss = d_model.train_on_batch(X_history, y_history)[1]
        # d_loss = d_model.train_on_batch(np.append(X_history, X_test, axis=0) , np.append(y_history, y_test, axis=0) )
        
        test_loss = d_model.evaluate(X_test, y_test, verbose=0)[1]
        test_losses[i] = test_loss
    if i % gen_mod == 0:    
        x_input = np.random.randn(batch_size, latent_dim)
        X = np.ones((batch_size, n_dipoles))
        d_model.trainable = False
        gan_loss = gan_model.train_on_batch(x_input, X)

    print(f'test_loss: {test_loss:.2f}, disc-loss: {d_loss:.2f}, gan-loss: {gan_loss:.2f}')
    d_losses[i] = d_loss
    
    gan_losses[i] = gan_loss
    

  0%|          | 0/1000 [00:00<?, ?it/s]

test_loss: -0.10, disc-loss: 0.00, gan-loss: 2.67
test_loss: -0.07, disc-loss: -0.09, gan-loss: 2.63
test_loss: -0.05, disc-loss: -0.06, gan-loss: 2.58
test_loss: -0.04, disc-loss: -0.04, gan-loss: 2.55
test_loss: -0.03, disc-loss: -0.03, gan-loss: 2.53
test_loss: -0.02, disc-loss: -0.01, gan-loss: 2.52
test_loss: -0.01, disc-loss: -0.00, gan-loss: 2.49
test_loss: -0.01, disc-loss: 0.01, gan-loss: 2.47
test_loss: 0.00, disc-loss: 0.02, gan-loss: 2.45
test_loss: 0.01, disc-loss: 0.03, gan-loss: 2.43
test_loss: 0.01, disc-loss: 0.04, gan-loss: 2.41
test_loss: 0.02, disc-loss: 0.05, gan-loss: 2.39
test_loss: 0.03, disc-loss: 0.05, gan-loss: 2.37
test_loss: 0.03, disc-loss: 0.06, gan-loss: 2.35
test_loss: 0.04, disc-loss: 0.07, gan-loss: 2.32
test_loss: 0.04, disc-loss: 0.08, gan-loss: 2.31
test_loss: 0.05, disc-loss: 0.09, gan-loss: 2.28
test_loss: 0.05, disc-loss: 0.10, gan-loss: 2.26
test_loss: 0.05, disc-loss: 0.10, gan-loss: 2.24
test_loss: 0.05, disc-loss: 0.11, gan-loss: 2.21
test_l

KeyboardInterrupt: 

In [222]:
%matplotlib qt
plt.figure()
plt.plot(d_losses, label="Discriminator Loss")
plt.plot(gan_losses, label="GAN Loss")
plt.plot(test_losses, label="Test Loss")
plt.legend()

# g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg)

y = generate_samples(g_model, 100, latent_dim)
data = abs(y).mean(axis=0)
stc_ = stc.copy()
stc_.data[:, 0] = data
stc_.plot(**plot_params)

stc_.data = y.T
stc_.plot(**plot_params)


plt.figure()
plt.hist(y.flatten())

(array([  761.,   616.,   962.,  3643., 57758., 58807.,  3663.,   893.,
          632.,   665.]),
 array([-0.7615942 , -0.60927534, -0.4569565 , -0.30463767, -0.15231884,
         0.        ,  0.15231884,  0.30463767,  0.4569565 ,  0.60927534,
         0.7615942 ], dtype=float32),
 <BarContainer object of 10 artists>)

Using control points [0.26009514 0.39334973 0.76121824]
Using control points [0.25965381 0.41853858 0.74863843]


In [219]:
# plt.figure()
# plt.plot(y[0])
l1_sparsity(y[0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.48456874>

# Evaluate

## Simulation

In [199]:
n_samples = 2
# settings = dict(duration_of_trial=0.1, extents=(1,40), number_of_sources=(1,15), target_snr=1e99, source_number_weighting=False)
# settings = dict(duration_of_trial=0.1, extents=(1,40), number_of_sources=15, target_snr=1e99)
settings = dict(duration_of_trial=0.01, extents=25, number_of_sources=1, target_snr=1e99)

sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X = np.stack([eeg.average().data for eeg in sim_test.eeg_data], axis=0)
y = np.stack([src.data for src in sim_test.source_data], axis=0)

X, y = prep_data(X[0].T, y[0].T)

y_hat = d_model.predict(X)
y_hat.shape
stc = sim_test.source_data[0]
stc.plot(**plot_params, brain_kwargs=dict(title="Ground Truth Sim"))

stc_hat = stc.copy()
stc_hat.data = y_hat.T
stc_hat.plot(**plot_params, brain_kwargs=dict(title="GAN"))
from scipy.stats import pearsonr
from esinet.evaluate import eval_auc
_, pos = util.unpack_fwd(fwd)[1:3]
auc = np.mean([np.mean(eval_auc(y_true, y_pred, pos)) for y_true, y_pred in zip(stc.data.T, stc_hat.data.T)])
cosine = tf.keras.losses.CosineSimilarity()(tf.cast(stc.data.T, dtype=tf.float32), tf.cast(stc_hat.data.T, dtype=tf.float32)).numpy()
r,p = pearsonr(stc.data.flatten(), stc_hat.data.flatten())
print(f'auc={auc:.2f}, cosine={cosine}, r={r:.2f}, p={p:.4f}\n')

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 392.38it/s]


auc=0.25, cosine=0.006478124298155308, r=-0.03, p=0.3583



Using control points [0.00000000e+00 0.00000000e+00 6.75949907e-08]
Using control points [0.00000000e+00 0.00000000e+00 6.75949907e-08]


In [20]:
sys.path.insert(0, '../../invert/')
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping
from invert.solvers.wrop import SolverLAURA
from invert.solvers.empirical_bayes import SolverChampagne

# solver = SolverLAURA().make_inverse_operator(fwd)
solver = SolverChampagne().make_inverse_operator(fwd)

stc_mne = solver.apply_inverse_operator(sim_test.eeg_data[0].average())
stc_mne.data = stc_mne.data / np.max(abs(stc_mne.data))
stc_mne.plot(**plot_params, brain_kwargs=dict(title=solver.name))
r,p = pearsonr(stc.data.flatten(), stc_mne.data.flatten())
auc = np.mean([np.mean(eval_auc(y_true, y_pred, pos)) for y_true, y_pred in zip(stc.data.T, stc_mne.data.T)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.66, r=0.54, p=0.0000



Traceback (most recent call last):
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\utils.py", line 60, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 731, in _clean
    self.clear_glyphs()
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1629, in clear_glyphs
    assert sum(len(v) for v in self.picked_points.values()) == 0
AssertionError


# Test Discriminator with Generator

In [157]:
# g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg)

y = generate_samples(g_model, 10000, latent_dim)
y.shape
data = abs(y).mean(axis=0)
stc_ = stc.copy()
stc_.data[:, 0] = data
stc_.plot(**plot_params)

plt.figure()
plt.hist(y.flatten())

(array([6.507000e+03, 2.830100e+04, 1.034900e+04, 7.477900e+04,
        6.305901e+06, 6.296174e+06, 7.593000e+04, 1.007600e+04,
        2.595100e+04, 6.032000e+03]),
 array([-0.7615942 , -0.60927534, -0.4569565 , -0.30463767, -0.15231884,
         0.        ,  0.15231884,  0.30463767,  0.4569565 ,  0.60927534,
         0.7615942 ], dtype=float32),
 <BarContainer object of 10 artists>)

In [223]:
# generated data:
y = generate_samples(g_model, 32, latent_dim).T
X = (leadfield @ y).T
X, y = prep_data(X,y)
y = y
X = X
y[np.isnan(y)] = 0
stc_hat.data = y
stc_hat.plot(**plot_params, brain_kwargs=dict(title="Ground Truth"), clim=dict(kind='value', pos_lims=(0, 0.5, 1)))

y_hat = d_model.predict(X)
y_hat[np.isnan(y_hat)] = 0

stc_hat.data = y_hat.T
stc_hat.plot(**plot_params, brain_kwargs=dict(title="GAN"))

r,p = pearsonr(y.flatten(), y_hat.T.flatten())
auc = np.mean([np.mean(eval_auc(yy_true, yy_pred, pos)) for yy_true, yy_pred in zip(y.T, y_hat)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.58, r=-0.71, p=0.0000



Traceback (most recent call last):
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\utils.py", line 60, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 731, in _clean
    self.clear_glyphs()
  File "c:\Users\lukas\virtualenvs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1629, in clear_glyphs
    assert sum(len(v) for v in self.picked_points.values()) == 0
AssertionError


# Linear Solution

In [150]:
sys.path.insert(0, '../../invert/')
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping
from invert.solvers.wrop import SolverLAURA

evoked = mne.EvokedArray(X[:, 0, :].T, info)

# solver = SolverLAURA().make_inverse_operator(fwd)
solver = SolverChampagne().make_inverse_operator(fwd)
stc_mne = solver.apply_inverse_operator(evoked)
stc_mne.data = stc_mne.data / np.max(abs(stc_mne.data))
stc_mne.plot(**plot_params, brain_kwargs=dict(title=solver.name))
r,p = pearsonr(y[:, 0, :].flatten(), stc_mne.data.flatten())
auc = np.mean([np.mean(eval_auc(yy_true, yy_pred, pos)) for yy_true, yy_pred in zip(y[:, 0, :].T, stc_mne.data .T)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.50, r=0.03, p=0.0000



Using control points [1. 1. 1.]
Using control points [1. 1. 1.]


Traceback (most recent call last):
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\utils.py", line 60, in safe_event
    return fun(*args, **kwargs)
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 731, in _clean
    self.clear_glyphs()
  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1629, in clear_glyphs
    assert sum(len(v) for v in self.picked_points.values()) == 0
AssertionError


## old functions

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda
import numpy as np


basic_sources = np.identity(n_dipoles)
basic_sources = laplace_operator @ basic_sources
basic_sources = np.concatenate([basic_sources, -1*basic_sources], axis=1)
basic_eeg = leadfield @ basic_sources
print(basic_sources.shape, basic_eeg.shape)

n_chans, n_dipoles = leadfield.shape

def define_generator(latent_dim):
    g_model = tf.keras.Sequential()
    input_shape = (None, latent_dim)
    g_model.add(InputLayer(input_shape=input_shape))
    g_model.add(Dense(latent_dim, name="HL1"))
    g_model.add(Dense(n_dipoles, name="Output"))
    # g_model.build()
    # g_model.compile(optimizer='adam', loss="mse")
    # g_model.summary()
    return g_model
    
def define_discriminator(hidden_units=100):
    input_shape = (None, n_chans)
    d_model = tf.keras.Sequential()
    d_model.add(InputLayer(input_shape=input_shape))
    d_model.add(Dense(hidden_units, name="HL1"))
    d_model.add(Dense(n_dipoles, name="Output"))
    d_model.build()
    d_model.compile(optimizer='adam', loss=tf.keras.losses.CosineSimilarity())
    # d_model.summary()
    return d_model

def define_gan(g_model, d_model, latent_dim):
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    d_model.trainable = False
    
    input_shape = (None, latent_dim)
    
    lam = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(g_model.output)
    print(lam)
    discriminator = d_model(lam)
    model = tf.keras.Model(inputs=g_model.input, outputs=[d_model.output, g_model.output], name='Contextualizer')


    # model = tf.keras.Sequential()
    # model.add(g_model)
    # model.add(Lambda(lambda x: tf.linalg.matmul(leadfield_, x)))
    # model.add(d_model)
    # model.compile(loss='binary_crossentropy', optimizer="adam")

    return model
 

